# Сравнение AdaBoost и Decision Trees на временных рядах цен на электричество

_Автор: Сергей Иванычев, 376 гр_

_sergeyivanychev@gmail.com_

Постановка вопроса:

```
Вопрос: что лучше: decision trees или adaboost в готовых версиях для матлаба для задачи по ссылке в постановке в прикрепленном файле, на этих данных, прогноз на сутки вперед. Качество прогноза MAPE усредненное по контрольной выборке
```

В связи с тем, что после обсуждения с Радославом Нейчевым поставленного вопроса, у исследователя не получилось найти готовых решений, реализовывающих на матлабе подобные алгоритмы было принято решение воспользоваться `Python 3` и, в частности, библиотекой `sklearn`. Приведем примерный ответ в этом документе.

## Импортируем библиотеки

In [42]:
import numpy as np
import pandas as pd
from pandas import DataFrame as df
from pandas import read_csv

import sklearn

import matplotlib.pyplot as pl
%matplotlib inline
import seaborn as sns

## Считываем данные

In [43]:
columns = []
for x in range (1, 25):
    columns.append("hour" + str(x))
pd_data = pd.read_csv("GermanSpotPrice.csv", names=columns)
pd_data.h

,hour1,hour2,hour3,hour4,hour5,hour6,hour7,hour8,hour9,hour10,...,hour15,hour16,hour17,hour18,hour19,hour20,hour21,hour22,hour23,hour24
731582,15.09,11.46,9.58,6.55,4.36,2.09,0.06,0.14,0.00,0.16,...,3.20,3.17,6.09,11.25,13.61,10.57,15.23,11.57,14.81,10.40
731583,7.94,4.40,2.05,1.98,2.08,6.08,9.58,15.45,16.63,21.33,...,25.03,21.85,26.38,32.28,34.54,31.42,27.66,19.93,23.98,17.14
731584,8.84,7.00,3.51,2.06,2.08,7.04,8.85,16.82,21.81,23.36,...,18.21,14.58,19.06,25.89,24.07,18.38,16.00,13.21,13.93,10.43
731585,15.34,8.04,7.05,6.54,4.88,4.09,2.02,7.05,10.55,20.16,...,19.24,18.59,23.19,28.98,33.43,27.76,24.46,20.14,23.88,18.93
731586,13.18,8.34,7.54,6.59,6.77,5.51,1.04,2.00,4.54,7.06,...,11.20,11.43,17.04,29.39,29.91,25.01,21.97,16.59,24.73,14.86


## Методы предсказания

Для предсказания цен на электричество на 24 часа вперед будем использовать указанные в письме методы из библиотеки `sklearn`. Здесь попробуем `AdaBoost`, `Decision Tree` и `Random Forest` для задачи регрессии. 

```
http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.AdaBoostRegressor.html
http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html
http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html
```

Для предпологаю, что цены на j-й час i-го дня зависят от цен за предыдущие `WINDOW` дней.
Для решения регрессии для i-го дня будем учитывать обучающую выборку за `PERIOD`

MAPE:

```
https://gist.github.com/amanahuja/6315882
```

In [44]:
from sklearn.ensemble import AdaBoostRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

WINDOW = 7
PERIOD = 30

In [46]:
data = pd_data.as_matrix()
data.shape
data[0:5, :]

array([[ 15.09,  11.46,   9.58,   6.55,   4.36,   2.09,   0.06,   0.14,
          0.  ,   0.16,   1.15,   1.92,   7.03,   5.09,   3.2 ,   3.17,
          6.09,  11.25,  13.61,  10.57,  15.23,  11.57,  14.81,  10.4 ],
       [  7.94,   4.4 ,   2.05,   1.98,   2.08,   6.08,   9.58,  15.45,
         16.63,  21.33,  25.03,  30.01,  31.83,  31.88,  25.03,  21.85,
         26.38,  32.28,  34.54,  31.42,  27.66,  19.93,  23.98,  17.14],
       [  8.84,   7.  ,   3.51,   2.06,   2.08,   7.04,   8.85,  16.82,
         21.81,  23.36,  24.09,  25.12,  23.64,  22.3 ,  18.21,  14.58,
         19.06,  25.89,  24.07,  18.38,  16.  ,  13.21,  13.93,  10.43],
       [ 15.34,   8.04,   7.05,   6.54,   4.88,   4.09,   2.02,   7.05,
         10.55,  20.16,  22.35,  26.48,  23.98,  22.23,  19.24,  18.59,
         23.19,  28.98,  33.43,  27.76,  24.46,  20.14,  23.88,  18.93],
       [ 13.18,   8.34,   7.54,   6.59,   6.77,   5.51,   1.04,   2.  ,
          4.54,   7.06,  12.48,  20.  ,  17.02,  14.42,  11.

### Что происходит

Предсказание цен на следующий за PERIOD дней день есть 24 задачи регрессии (по одной на час). Обучающая выборка для i-й задачи регрессии $$X^l = (X, y)_{i=1}^l\;\; l = PERIOD - WINDOW$$ где $y_j$ - цена за электричество в $j+1$-й день а $X_j$ -  матрица размерности $WINDOW\times24$. То есть объектом для $i$-го часа считается $WINDOW$ предыдущих дней.

In [1]:
def get_train_answer_sets(data, day_num):
    """
                    Extracts training and testing sets

    Input:          data        whole DAYS x 24 numpy matrix
                    day_num     number of day in [WINDOW, DAYS) which is observed
    External:       PERIOD      the period of time we take into account
                    WINDOW      if price in j-th hour of i-th day is answer
                                then [i-WINDOW,i) days are object
    Return:        (X_train, Y_train, X_test, Y_test)
                    X_train     array of (PERIOD - WINDOW) (WINDOW, 24) matrices
                    Y_train     array of (PERIOD - WINDOW) 24 sample vectors
                    X_test      (WINDOW, 24) matrix
                    Y_tess      24 sample vector
    Note:           (PERIOD - WINDOW) is the length of training set

    """
    set_len = PERIOD-WINDOW
    Y = np.zeros((set_len + 1, 24))
    X = np.zeros((set_len + 1, WINDOW, 24))
    for i in range(set_len + 1):
        day = day_num - set_len + i
        Y[i] = data[day]
        X[i] = data[day - WINDOW:day, :]
    X_current = X[set_len, :, :]
    Y_current = Y[set_len, :]
    return (X[:set_len, :, :], Y[:set_len, :], X_current, Y_current)

def get_regressors(X_all, Y_all):
    """         
                Builds regressors of all types for each hour

    Input:      X_all, Y_all        training set
    Return:    (ada_list, tree_list, forest_list)
                ada_list            list of 24 AdaBoost regressors
                tree_list           DecisionTree
                forest_list         RandomForest
    """
    ada = []
    tree = []
    forest = []
    ada_temp = None
    tree_temp = None
    forest_temp = None
    for hour in range(24):
        X = X_all.reshape((PERIOD-WINDOW), WINDOW * 24)
        Y = Y_all[:, hour]
        ada_temp = AdaBoostRegressor(DecisionTreeRegressor())
        tree_temp = DecisionTreeRegressor()
        forest_temp = RandomForestRegressor()
        ada_temp.fit(X, Y)
        tree_temp.fit(X, Y)
        forest_temp.fit(X, Y)
        ada.append(ada_temp)
        tree.append(tree_temp)
        forest.append(forest_temp)
    return (ada, tree, forest)

def get_forecast(ada_list, tree_list, forest_list, X):
    """
                Predicting using trained regressors

    Input:      ada_list
                tree_list
                forest_list         three 24-len lists with regressors
    Output:     three 24-len float vectors with prediction 
    """
    ret = np.zeros((3, 24))
    for idx, regr in enumerate([ada_list, tree_list, forest_list]):
        for hour in range(24):
            ret[idx, hour] = regr[hour].predict(X.reshape(1, -1))
    return (ret[0],  ret[1], ret[2])

def mape(y_true, y_pred):
    """
                Mean absolute percentage error
    """
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100



In [124]:
begin = 500
end = 550
ada_errors, tree_errors, forest_errors = [], [], []
for i in range(begin, end):
    X_train, Y_train, X_test, Y_test = get_train_answer_sets(data, i)    # getting training sets
    ada_list, tree_list, forest_list = get_regressors(X_train, Y_train)  # generating regressors
    ada_fcst, tree_fcst, forest_fcst = get_forecast(ada_list, tree_list, forest_list, X_test) #forcasting
    ada_error = mape(Y_test, ada_fcst)
    tree_error = mape(Y_test, tree_fcst)
    forest_error = mape(Y_test, forest_fcst)     # calculating quality
    ada_errors.append(ada_error)
    tree_errors.append(tree_error)
    forest_errors.append(forest_error)
#     print(str(i) + " DONE")

ada_mean_error = sum(ada_errors) / float(len(ada_errors))
tree_mean_error = sum(tree_errors) / float(len(tree_errors))
forest_mean_error = sum(forest_errors) / float(len(forest_errors))
print("AdaBoost mean error: %f" % ada_mean_error)
print("Decision Tree mean error: %f" % tree_mean_error)
print("RandomForest mean error: %f" % forest_mean_error)
    
    

AdaBoost mean error: 27.870050
Decision Tree mean error: 34.414409
RandomForest mean error: 30.071874


### Заключение

Возможно, в решении допущены ошибки, так как результаты на, например, `begin = 100, end = 105` и `begin = 105, end = 110` довольно сильно различаются. В целом можно заметить, что при стандартных параметрах `sklearn` в среднем чуть лучше себя показывает `AdaBoost` с небольшим отрывом.